## 1: The Competition
We'll be learning how to generate a submission for a Kaggle competition. Kaggle is a site where you create algorithms, and compete against machine learning practitioners around the world. Your algorithm wins if it's the most accurate on a given dataset. Kaggle is a fun way to practice your machine learning skills.

Kaggle has several different competitions on their site. One of them is about predicting which passengers survived the sinking of the Titanic. In this notebook, we'll explore the data, train our first model, and prepare our first submission to the competition.

Let's take a look at the dataset. Each row represents a passenger on the Titanic, and some information about them. 

- PassengerId -- A numerical id assigned to each passenger.
- Survived -- Whether the passenger survived (1), or didn't (0). We'll be making predictions for this column.
- Pclass -- The class the passenger was in -- first class (1), second class (2), or third class (3).
- Name -- the name of the passenger.
- Sex -- The gender of the passenger -- male or female.
- Age -- The age of the passenger. Fractional.
- SibSp -- The number of siblings and spouses the passenger had on board.
- Parch -- The number of parents and children the passenger had on board.
- Ticket -- The ticket number of the passenger.
- Fare -- How much the passenger paid for the ticker.
- Cabin -- Which cabin the passenger was in.
- Embarked -- Where the passenger boarded the Titanic.

A good first step is to think logically about the columns and what we're trying to predict. What variables might logically affect the outcome of survived?

We know that women and children were more likely to survive. Thus, Age and Sex are probably good predictors. It's also logical to think that passenger class might affect the outcome, as first class cabins were closer to the deck of the ship. Fare is tied to passenger class, and will probably be highly correlated with it, but might add some additional information. Number of siblings and parents/children will probably be correlated with survival one way or the other, as either there are more people to help you, or more people to think about and try to save.

There's a less clear link between survival and columns like Embarked (maybe there is some information about how close to the top of the ship people's cabins were here), Ticket, and Name.

This step is generally known as acquiring domain knowledge, and it fairly important to most machine learning tasks. We're looking to engineer the features so that we maximize the information we have about what we're trying to predict.

## 2: Looking At The Data
We'll be using the pandas library, and scikit-learn to analyze our data and create a submission. 
A good second step is to look at high level descriptors of the data. In this case, we can use the pandas .describe() method to look at different characteristics of each numeric column.

In [3]:
# import packages
import pandas as pd
import numpy as np


In [4]:
# import your dataset
# We can use the pandas library in python to read in the csv file.
# This creates a pandas dataframe and assigns it to the titanic variable.
titanic = pd.read_csv('Data/titanic_train.csv').dropna()

# Print the first 5 rows of the dataframe.
print(titanic.head(5))

# describe the dataset
print(titanic.describe())


    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name     Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                             McCarthy, Mr. Timothy J    male  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                           Bonnell, Miss. Elizabeth  female  58.0      0   

    Parch    Ticket     Fare Cabin Embarked  
1       0  PC 17599  71.2833   C85        C  
3       0    113803  53.1000  C123        S  
6       0     17463  51.8625   E46        S  
10      1   PP 9549  16.7000    G6        S  
11      0    113783  26.5500  C103        S  
       PassengerId    Survived  

## 3: Missing Data
When you used .describe() on the titanic dataframe in the last screen, you might have noticed that the Age column has a count of 714 when all the other columns have a count of 891. This indicates that there are missing values in the Age column -- the count is of non-missing (null, NA, or not a number) values.

This means that the data isn't perfectly clean, and we're going to have to clean it ourselves. We don't want to have to remove the rows with missing values, because more data helps us train a better algorithm. We also don't want to get rid of the whole column, as age is probably fairly important to our analysis.

There are many strategies for cleaning up missing data, but a simple one is to just fill in all the missing values with the median of all the values in the column.

We can select a single column by indexing the dataframe like a dictionary. This gives us a pandas series:


titanic["Age"]
We can then use the .fillna method on the series to replace any missing values. .fillna takes one argument, the value to replace the missing values with.

In our case, we want to fill with the median of the column:


.fillna(titanic["Age"].median())
We then have to assign the result back to the column to replace it:


titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())

In [6]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())

## 4: Non-Numeric Columns
When we used .describe(), you might have also noticed that not all the columns were shown. Only the numeric columns were shown. Several of our columns are non-numeric, which is a problem when it comes time to make predictions -- we can't feed non-numeric columns into a machine learning algorithm and expect it to make sense of them.

We have to either exclude our non-numeric columns when we train our algorithm (Name, Sex, Cabin, Embarked, and Ticket), or find a way to convert them to numeric columns.

We'll ignore the Ticket, Cabin, and Name columns. There isn't much information we can extract from there. Most of the values in the cabin column are missing (only 204 values out of 891 rows), and it likely isn't a particularly informative column in the first place. The Ticket and Name columns are unlikely to tell us much without some domain knowledge about what the ticket numbers mean, and about which names correlate with characteristics like large or rich families.

## 5: Converting The Sex Column
The Sex column is non-numeric, but we want to keep it around -- it could be very informative. We can convert it to a numeric column by replacing each gender with a numeric code. A machine learning algorithm will then be able to use these categories to make predictions.

To do this, we first have to find all the unique genders in the column (we know male and female are there, but did whoever recorded the dataset use another code for missing values?). We'll also assign a code of 0 to male, and a code of 1 to female.

We can select all the male values in the Sex column with:


titanic.loc[titanic["Sex"] == "male", "Sex"]

We can then replace all these values with 0:


titanic.loc[titanic["Sex"] == "male", "Sex"] = 0

In [7]:
# Find all the unique genders -- the column appears to contain only male and female.
print(titanic["Sex"].unique())

# Replace all the occurences of male with the number 0.
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1

['female' 'male']


## 6: Converting The Embarked Column
We now can convert the Embarked column to codes the same way we converted the Sex column. The unique values in Embarked are S, C, Q, and missing (nan). Each letter is an abbreviation of an embarkation port name.

In [8]:
# Find all the unique values for "Embarked".
print(titanic["Embarked"].unique())
titanic["Embarked"] = titanic["Embarked"].fillna("S")

titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

['C' 'S' 'Q']


## 7: Cross Validation
Below, we will use cross validation approach to build up and select models. 

In [23]:
# define the X and y variables

from sklearn.cross_validation import train_test_split

predictors = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

X = titanic[predictors]
y = titanic["Survived"]

# Instruction: change the value of random_state
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=0)


In [26]:
# Instruction: fit the model using X_train and y_train

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=6)

clf.fit(X_train,y_train)

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.73


In [25]:
# View a list of the features and their importance scores
print(list(zip(titanic[predictors], clf.feature_importances_)))

[('Pclass', 0.03508613196339947), ('Sex', 0.37486751476415575), ('Age', 0.5809902919463712), ('SibSp', 0.0), ('Parch', 0.009056061326073698), ('Embarked', 0.0)]


In [ ]:
# create a random forest model





## 8: Processing The Test Set
Our accuracy is decent, but not great. You may try a few things to make it better.

Now, we need to make a submission to the competition. To do this, we need to take the exact same steps on the test data that we took on the training data. If we don't do the exact same operations, then we won't be able to make valid predictions on it.

These operations are all the changes we made to the columns before.

In [28]:
titanic_test = pd.read_csv("Data/titanic_test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic["Age"].median())
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2

## 9.Generating A Submission File
Now we have everything we need to generate a submission for the competition!

First, we have to train an algorithm on the training data. Then, we make predictions on the test set. Finally, we'll generate a csv file with the predictions and passenger ids.

In [29]:
# Make predictions using the test set.
predictions = clf.predict(titanic_test[predictors])

# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })

submission.to_csv("Data/kaggle.csv", index=False)